In [3]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler # Scale data
from sklearn.model_selection import train_test_split #split data into test and train
import matplotlib.pyplot as plt

In [9]:
# Import data from sklearn
bc = datasets.load_breast_cancer()
x, y = bc.data, bc.target

n_samples, n_features = x.shape

# Split data
x_test, x_train, y_test, y_train = train_test_split(x, y, test_size=0.2, random_state=1)

# Scale features - 0 mean and unit variance
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

# convert to tensors
x_train = torch.from_numpy(x_train.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
x_test = torch.from_numpy(x_test.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [11]:
# design the model - sigmoid(wx + b)
class Model(nn.Module):
  def __init__(self, input_features):
    super(Model, self).__init__()
    self.linear = nn.Linear(input_features, 1)
  
  def forward(self, x):
    y_predicted = torch.sigmoid(self.linear(x))
    return y_predicted

model = Model(n_features)

# define the loss (criterion)
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

n_epochs = 100

In [13]:
for epoch in range(n_epochs):
  # forward pass
  y_pred = model(x_train)

  # loss 
  loss = criterion(y_pred, y_train)

  # backward pass / calculate the gradients
  loss.backward()

  # update the weights
  optimizer.step()
  optimizer.zero_grad()

  if (epoch+1) % 10 == 0:
    print(f'Epoch {epoch+1}; loss: {loss.item():.4f} ; ')

# plot - This will not modify the gradient calculation function
with torch.no_grad():
  y_predicted  = model(x_test)
  y_predicted_cls = y_predicted.round()
  acc = y_predicted_cls.eq(y_test).sum() / y_test.shape[0]
  print(acc)

Epoch 10; loss: 0.7355 ; 
Epoch 20; loss: 0.6101 ; 
Epoch 30; loss: 0.5290 ; 
Epoch 40; loss: 0.4719 ; 
Epoch 50; loss: 0.4291 ; 
Epoch 60; loss: 0.3957 ; 
Epoch 70; loss: 0.3686 ; 
Epoch 80; loss: 0.3462 ; 
Epoch 90; loss: 0.3272 ; 
Epoch 100; loss: 0.3108 ; 
tensor(0.9363)
